<a href="https://colab.research.google.com/github/KeepSOBP/dame/blob/master/dame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dame dane~ dame yo~

20200901 code version


Edited by [KeepSOBP](https://github.com/KeepSOBP/dame)
Original by [AliaksandrSiarohin](https://github.com/AliaksandrSiarohin/first-order-model)

In [ ]:
%%shell
apt install unzip
git clone https://github.com/AliaksandrSiarohin/first-order-model
wget https://github.com/KeepSOBP/dame/releases/download/damesource/damesource.zip
unzip damesource.zip -d /content/damesource

In [ ]:
cd first-order-model

In [ ]:
#@markdown <h3><- Please make sure rename image name to 02.png! and resize to 256x256 too.</h3>

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Upload complete!'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
import warnings
from demo import load_checkpoints
print("Working!")
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/first-order-model/02.png')
driving_video = imageio.mimread('/content/damesource/04.mp4')

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/damesource/vox-cpk.pth.tar')
print("Done")

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../complete.mp4', [img_as_ubyte(frame) for frame in predictions])
files.download('../complete.mp4')